In [1]:
import math
import numpy as np
import pandas as pd
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS

In [2]:
def read_stop_words(file):
    with open(file) as f:
        stop_words = f.read().split('\n')

    return stop_words

In [3]:
df = pd.read_csv("./data/reviews.csv")

In [4]:
df.head()

,Unnamed: 0,title,pos_text,neg_text,ratingValue,bestRating
0,0,Ідеально для ділової поїздки!,Ідеально для ділової поїздки! Господар зустрів...,NaN,10.0,10.0
1,1,"Затишний, чистий номер з усіма зручностями.","Затишний, чистий номер з усіма зручностями. Чу...","При бронюванні вказала час прибуття о 7 ранку,...",9.2,10.0
2,2,Все сподобалося. Рекомендую,"Чисто, тихо, комфортно. Зустрів і провів приєм...",На барі кава тільки 3 в 1. Хотілося звичайної ...,9.6,10.0
3,3,"Зручне розташування,чудовий вигляд з вікна,в н...","Зручне розташування,чудовий вигляд з вікна,в н...",NaN,10.0,10.0
4,4,"Все чудово: 9,9 балів!","Нові апартаменти на останньому поверсі ЖК, біл...","Немає терміналу для оплати кредиткою, тільки г...",10.0,10.0


In [6]:
df['pos_text'][1]

'Затишний, чистий номер з усіма зручностями. Чудовий краєвид з вікна. Є можливість підігріти їжу та зробити чай/каву.'

### 2. Mutual information

**Mutual information tells you how much you learn about X from knowing the value of Y (on average over the choice of Y).** 


Since we found the word frequency is not a good indicator for the sentiment analysis, we will examine *mutual information*  for an alternative metric.

http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mutual_info_score.html

In [7]:
math.isnan(df['neg_text'].values[0])

True

In [8]:
def is_review_valid(review):
    try:
        if len(review) == 0:
            return False
        return True
    except:
        return False

In [9]:
def get_pos_neg_review(df):
    gt_score, reviews = [], []
    for i in range(0, len(df)):
        if is_review_valid(df['pos_text'][i]):
            reviews.append(df['pos_text'][i])
            gt_score.append(1)
            
        if is_review_valid(df['neg_text'][i]):
            reviews.append(df['neg_text'][i])
            gt_score.append(0)
            
    return reviews, gt_score

In [10]:
reviews, gt_score = get_pos_neg_review(df)

In [11]:
uk_stop_words = read_stop_words('./data/ukrainian-stopwords.txt')

In [12]:
# let's calculate Mutual Information for unigrams and bigrams
vectorizer = CountVectorizer(ngram_range=(1,1), stop_words=uk_stop_words, max_features=500)
countvector = vectorizer.fit_transform(reviews)
densevector = np.array(countvector.todense())
    
# miScore_unigram = pd.DataFrame(data = {'word': vectorizer.get_feature_names(),
#              'MI Score': [mutual_info_score(gtScore, densevector[:,i].squeeze()) for i in range(500)]})

miScore_unigram = pd.DataFrame(data =
                               {'MI Score': [mutual_info_score(gt_score, densevector[:,i].squeeze()) for i in range(500)]}
                              , index = vectorizer.get_feature_names())

# Bigram version
vectorizer = CountVectorizer(ngram_range=(2,2), stop_words=uk_stop_words, max_features=500)
countvector = vectorizer.fit_transform(reviews)
densevector = np.array(countvector.todense())
miScore_bigram = pd.DataFrame(data =
                    {'MI Score': [mutual_info_score(gt_score, densevector[:,i].squeeze()) for i in range(500)]},
                    index = vectorizer.get_feature_names())

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [13]:
miScore_unigram.sort_values('MI Score', inplace=True, ascending=False)
print('Mutual Information - Unigram')
miScore_unigram.head(10)

Mutual Information - Unigram


,MI Score
персонал,0.077176
розташування,0.076582
привітний,0.046049
чисто,0.042327
зручне,0.030128
відсутність,0.016802
чудове,0.014197
центрі,0.013936
гарний,0.013700
погана,0.013699


In [14]:
miScore_bigram.sort_values('MI Score', inplace=True, ascending=False)
print('Mutual Information - Bigram')
miScore_bigram.head(10)

Mutual Information - Bigram


,MI Score
привітний персонал,0.033663
зручне розташування,0.017697
місце розташування,0.009666
приємний персонал,0.009136
погана звукоізоляція,0.007070
чудове розташування,0.006828
інтер єр,0.006487
центрі міста,0.006430
персонал привітний,0.006034
ванній кімнаті,0.005576


###  3. Pointwise Mutual Information

In [15]:
def getPMI_ngram(reviews, gt_score, gt, ngram_range=(1,1), max_features=500):
    print(reviews)
    vectorizer = CountVectorizer(ngram_range=ngram_range, stop_words='english', max_features=max_features)
    countvector = vectorizer.fit_transform(reviews)
    densevector = np.array(countvector.todense())
    
    px = sum(gt_score == gt) / len(reviews)
    pmis = []
    
    for i in range(max_features):
        py = sum(densevector[:,i] == 1) / len(reviews)
        pxy = sum((gt_score== gt) & (densevector[:,i] == 1)) / len(reviews)
        
        if pxy == 0:
            pmi = math.log10((pxy + 0.0001) / (px * py))
        else:
            pmi = math.log10(pxy / (px * py))
            
        pmis.append(pmi)
        
    gt_name = 'positive' if gt == 1 else 'negative'
    pmis = pd.DataFrame(data = {'pmi' + gt_name: pmis}, index = vectorizer.get_feature_names())
    return pmis.sort_values('pmi' + gt_name, ascending=False)

In [16]:
gt_score = np.array(gt_score)

In [87]:
pmiPos_unigram = getPMI_ngram(reviews, gt_score,  1)
pmiNeg_unigram = getPMI_ngram(reviews, gt_score,  0)
pmiPos_bigram = getPMI_ngram(reviews, gt_score,  1, ngram_range=(2,2))
pmiNeg_bigram = getPMI_ngram(reviews, gt_score,  0, ngram_range=(2,2))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Let's see what PMI values tell us about the reviews

In [31]:
print('PMI for positive reviews - Unigram')
pmiPos_unigram.head(10)

PMI for positive reviews - Unigram


,pmipositive
відмінне,0.241606
прекрасне,0.241606
простора,0.238721
доброзичливий,0.238471
просторий,0.237894
чудове,0.237260
уважний,0.236556
затишна,0.236074
гарне,0.233970
ідеальне,0.232891


In [32]:
print('PMI for positive reviews - Bigram')
pmiPos_bigram.head(10)

PMI for positive reviews - Bigram


,pmipositive
затишно чисто,0.241606
комфортний номер,0.241606
тихе місце,0.241606
зручний номер,0.241606
чудовий сніданок,0.241606
чудовий персонал,0.241606
вічливий персонал,0.241606
гарне розташування,0.241606
гарний сніданок,0.241606
та ввічливий,0.241606


In [33]:
print('PMI for negative reviews - Unigram')
pmiNeg_unigram.head(10)

PMI for negative reviews - Unigram


,pminegative
неприємний,0.366348
погана,0.363971
відсутня,0.358464
відсутній,0.357119
неможливо,0.355653
брудно,0.355335
погано,0.354805
тонкі,0.352569
шумоізоляція,0.342176
відсутність,0.342003


In [34]:
print('PMI for negative reviews - Bigram')
pmiNeg_bigram.head(20)

PMI for negative reviews - Bigram


,pminegative
погана шумоізоляція,0.369893
дуже тонкі,0.369893
дуже погана,0.369893
погана звукоізоляція,0.366016
неприємний запах,0.365697
чути все,0.363903
запах каналізації,0.362068
на підлогу,0.361119
чути як,0.360938
тонкі стіни,0.358804


In [35]:
pmiNeg_bigram.tail(10)

,pminegative
розташування самому,-1.671499
приємний персонал,-1.684145
дуже гарний,-1.687012
персонал чистота,-1.709288
зручне розташування,-1.720516
номері чисто,-1.727017
розташування привітний,-1.747378
чисто охайно,-1.763646
чистий номер,-1.950253
чудове розташування,-2.139309


In [36]:
pmiPos_bigram.tail(10)

,pmipositive
дуже погана,-1.247985
не можливо,-1.276908
тонкі стіни,-1.356819
чути як,-1.448590
погана шумоізоляція,-1.450728
на підлогу,-1.457364
запах каналізації,-1.506582
чути все,-1.621717
неприємний запах,-1.775427
погана звукоізоляція,-1.809546


In [38]:
df['pos_text'][137]

'Всі фотографії відповідають реальності. Шторка, робоча розетка, лампа. Зручне ліжко, новий посуд і чисті ванні кімнати.Ремонт новий. Адміністратори дуже приємні і виконують свою роботу швидко та якісно. Враження най-найкращі.'

In [39]:
data = ['Було колись, колись, , , , , , і де було']

In [40]:
pmiPos_bigram2 = getPMI_ngram(data, gt_score[:1],  1, max_features=4, ngram_range=(2,2))

['Було колись, колись, , , , , , і де було']


In [41]:
pmiPos_trigram = getPMI_ngram(reviews, gt_score,  1, ngram_range=(3,3))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [42]:
pmiPos_trigram.head(10)

,pmipositive
необхідне для проживання,0.241606
чисто зручні ліжка,0.241606
гарне розташування готелю,0.241606
комфортний чистий номер,0.241606
кухня де можна,0.241606
кухні все необхідне,0.241606
ліжко дуже зручне,0.241606
привітні дівчата на,0.241606
все як на,0.241606
все що треба,0.241606


In [60]:
pmiPos_trigram.index.name = 'ngram'

In [61]:
pmiPos_trigram

,score
ngram,
необхідне для проживання,0.241606
чисто зручні ліжка,0.241606
гарне розташування готелю,0.241606
комфортний чистий номер,0.241606
кухня де можна,0.241606
кухні все необхідне,0.241606
ліжко дуже зручне,0.241606
привітні дівчата на,0.241606
все як на,0.241606


## Save pmi results to file

In [62]:
pmiPos_trigram.to_csv('./data/trigram-pmi-positive-scores.csv')

In [71]:
pmiPos_bigram.index.name = 'ngram'
pmiPos_bigram = pmiPos_bigram.rename(columns={"pmipositive" : "score"})


In [72]:
pmiPos_bigram.to_csv('./data/bigram-pmi-positive-scores.csv')

In [73]:
pmiPos_unigram.index.name = 'ngram'
pmiPos_unigram = pmiPos_unigram.rename(columns={"pmipositive" : "score"})

In [74]:
pmiPos_unigram.to_csv('./data/unigram-pmi-positive-scores.csv')

In [80]:
pmiNeg_unigram.index.name = 'ngram'
pmiNeg_unigram = pmiNeg_unigram.rename(columns={"pminegative" : "score"})

In [82]:
pmiNeg_unigram.to_csv('./data/unigram-pmi-negative-scores.csv')

In [88]:
pmiNeg_bigram.index.name = 'ngram'
pmiNeg_bigram = pmiNeg_bigram.rename(columns={"pminegative" : "score"})

In [90]:
pmiNeg_bigram.to_csv('./data/bigram-pmi-negative-scores.csv')

In [98]:
pmiNeg_trigram = getPMI_ngram(reviews, gt_score, 0, ngram_range=(3,3))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [99]:
pmiNeg_trigram.index.name = 'ngram'
pmiNeg_trigram = pmiNeg_trigram.rename(columns={"pminegative" : "score"})

In [101]:
pmiNeg_trigram.to_csv('./data/trigram-pmi-negative-scores.csv')

In [102]:
pmiNeg_trigram

,score
ngram,
на сайті не,0.369893
єдине що не,0.369893
залишає бажати кращого,0.369893
це не дуже,0.369893
було гарячої води,0.369893
що відбувається коридорі,0.369893
не сподобалося те,0.369893
сніданок міг би,0.369893
могла бути кращою,0.369893
